In [ ]:
import os

import librosa
import numpy as np
import pandas as pd
import scipy.signal
from tqdm import tqdm

#path stuff:
import sys
sys.path.append('/Users/rileyedmunds/echonet/echonet')

from echonet.datasets.dataset import Dataset
from echonet.utils.generics import load_audio, to_one_hot


class OriginalESC(Dataset):
    """

    """
    def __init__(self, data_dir, work_dir, train_folds, validation_folds, test_folds, esc10=False):
        super().__init__(data_dir, work_dir)

        self.meta = pd.read_csv(data_dir + 'esc50.csv')

        self.train_folds = train_folds
        self.validation_folds = validation_folds
        self.test_folds = test_folds

        self.class_count = 50

        self.bands = 60
        self.segment_length = 101

        self.esc10 = esc10
        if self.esc10:
            self.class_count = 10
            self.meta = self.meta[self.meta['esc10']]
            self.categories = pd.unique(self.meta.sort_values('target')['category'])
            self.meta['target'] = self.to_targets(self.meta['category'])
        else:
            self.categories = pd.unique(self.meta.sort_values('target')['category'])

        self.train_meta = self.meta[self.meta['fold'].isin(self.train_folds)]
        self.validation_data.meta = self.meta[self.meta['fold'].isin(self.validation_folds)]
        self.test_data.meta = self.meta[self.meta['fold'].isin(self.test_folds)]

        self._validation_size = len(self.validation_data.meta)
        self._test_size = len(self.test_data.meta)

        self._generate_spectrograms()
        print("generated spectrograms")
        self._populate(self.validation_data)
        print("populated validation data")
        self._populate(self.test_data)
        print("populated test data")
        # self._populate_portion(self.test_data)



        #attempt to populate training data:
        
    def _generate_spectrograms(self):
        for row in tqdm(self.meta.itertuples(), total=len(self.meta)):
            specfile = self.work_dir + row.filename + '.orig.spec.npy'.format(self.bands)

#             if os.path.exists(specfile):
#                 continue

#             print("go..")
            audio = load_audio(self.data_dir + 'audio/' + row.filename, 22050)
            audio *= 1.0 / np.max(np.abs(audio))

            #real    
#             spec = librosa.feature.melspectrogram(audio, sr=22050, n_fft=1024,
#                                                   hop_length=512, n_mels=self.bands)
#             spec = librosa.logamplitude(spec)
#             print(spec)

            #complex
            spec = librosa.core.stft(audio, n_fft=1024, hop_length=512, dtype=np.complex64)
                #n_fft 1024 before, now 118 so that we have 60 bands
            
            # save_graph(spec)


            np.save(specfile, spec, allow_pickle=False)

    def _populate(self, data):
        X, y, meta = [], [], []

        for row in data.meta.itertuples():
            segments = self._extract_all_segments(row.filename) #generates deltas and gets data out of npy to pandas dataframe
            print(X)
            X.extend(segments)
            y.extend(np.repeat(row.target, len(segments)))
            values = dict(zip(row._fields[1:], row[1:]))
            columns = row._fields[1:]
            rows = [pd.DataFrame(values, columns=columns, index=[0]) for _ in range(len(segments))]
            meta.extend(rows)

        X = np.stack(X)
        y = to_one_hot(np.array(y), self.class_count)
        meta = pd.concat(meta, ignore_index=True)

        if self.data_mean is None:
            self.data_mean = np.mean(X)
            self.data_std = np.std(X)

        X -= self.data_mean
        X /= self.data_std

        data.X = X
        data.y = y
        data.meta = meta


    def _extract_all_segments(self, filename):
        spec = np.load(self.work_dir + filename + '.orig.spec.npy')

        segments = []
        hop_length = self.segment_length // 5
        offset = 0

        while offset < np.shape(spec)[1] - self.segment_length:
            segment = spec[:, offset:offset + self.segment_length]
            delta = self._generate_delta(segment)
            offset += hop_length
            segments.append(np.stack([segment, delta])) #segment.real

        return segments


    def _extract_segment(self, filename):
        spec = np.load(self.work_dir + filename + '.orig.spec.npy')
        offset = self.RandomState.randint(0, np.shape(spec)[1] - self.segment_length + 1)
        spec = spec[:, offset:offset + self.segment_length]
        delta = self._generate_delta(spec)
        return np.stack([spec, delta])

    def _generate_delta(self, spec):
        # ported librosa v0.3.1. implementation
        window = np.arange(4, -5, -1)
        padding = [(0, 0), (5, 5)]
        delta = np.pad(spec, padding, mode='edge')
        delta = scipy.signal.lfilter(window, 1, delta, axis=-1)
        idx = [Ellipsis, slice(5, -5, None)]
        return delta[idx]
        





    @property
    def input_shape(self):
        return 2, self.bands, self.segment_length

    @property
    def train_size(self):
        return len(self.train_meta)

    @property
    def validation_size(self):
        return self._validation_size

    @property
    def validation_segments(self):
        return len(self.validation_data.meta)

    @property
    def test_size(self):
        return self._test_size

    @property
    def test_segments(self):
        return len(self.test_data.meta)

    def to_categories(self, targets):
        return self.categories[targets]

    def to_targets(self, categories):
        return [np.argmax(self.categories == name) for name in categories]

    def test(self, model):
        return self._score(model, self.test_data)

    def validate(self, model):
        return self._score(model, self.validation_data)

    def iterbatches(self, batch_size):
        itrain = super()._iterrows(self.train_meta)

        while True:
            X, y = [], []

            for i in range(batch_size):
                row = next(itrain)
                X.append(self._extract_segment(row.filename))
                y.append(row.target)

            X = np.stack(X)
            y = to_one_hot(np.array(y), self.class_count)

            X -= self.data_mean
            X /= self.data_std

            yield X, y



    #beyond here is for training, and not used for data creation-----------

    def _score(self, model, data):
        predictions = pd.DataFrame(model.predict(data.X))
        results = pd.concat([data.meta[['filename', 'target']], predictions], axis=1)
        results = results.groupby('filename').aggregate('mean').reset_index()
        results['predicted'] = np.argmax(results.iloc[:, 2:].values, axis=1)
        return np.sum(results['predicted'] == results['target']) / len(results)


In [ ]:
####REAL


# Paper source code ported to Keras with some small adjustments.

# Reference:

# - [Environmental Sound Classification with Convolutional Neural Networks -
#     paper replication data](https://github.com/karoldvl/paper-2015-esc-convnet)


import argparse
import functools
import os
import sys

import numpy as np

import sys
print(sys.path)

#path stuff:
import sys
sys.path.append('/Users/rileyedmunds/echonet/echonet')

# if __name__ == '__main__':
# parser = argparse.ArgumentParser()
# parser.add_argument('--device', help='Theano device used for computations')
# args = parser.parse_args()

RANDOM_SEED = 20161013
np.random.seed(RANDOM_SEED)

# DEVICE = args.device if args.device else 'gpu0'
THEANO_FLAGS = ('device={},'
                'floatX=float32,'
                'dnn.conv.algo_bwd_filter=deterministic,'
                'dnn.conv.algo_bwd_data=deterministic').format('gpu0')
os.environ['THEANO_FLAGS'] = THEANO_FLAGS
os.environ['KERAS_BACKEND'] = 'theano'

# sys.path.append(os.path.abspath(os.path.dirname(__file__)) + '/..')
# sys.path.append(os.path.abspath(os.path.dirname('/Users/rileyedmunds/echonet/echonet/examples')) + '/..')

import keras
# keras.backend.set_image_dim_ordering('th')
# from keras.layers.convolutional import Convolution2D as Conv
# from keras.layers.convolutional import MaxPooling2D as Pool
# from keras.layers.core import Activation, Dense, Dropout, Flatten

from echonet.models import EchoNet
# from echonet.datasets.esc_original_real import OriginalESC as OriginalESC_real
# from echonet.datasets.esc_original import OriginalESC as OriginalESC




# def uniform(scale):
#     return functools.partial(keras.initializations.uniform, scale=scale)

# def normal(stdev):
#     return functools.partial(keras.initializations.normal, scale=stdev)

# TRAIN_FOLDS = [2, 3, 4]
    # VALIDATION_FOLDS = [5]
    # TEST_FOLDS = [1]

    #Note using validation as training and test as test.
TRAIN_FOLDS = [2, 3, 4]
VALIDATION_FOLDS = [2, 3, 4]
TEST_FOLDS = [1]

print('\nLoading ESC-50 dataset from ../data/ESC-50/')
# esc50_real = OriginalESC_real('../data/ESC-50/', '../data/.ESC-50.cache', TRAIN_FOLDS, VALIDATION_FOLDS,
#                     TEST_FOLDS)
esc50 = OriginalESC('../data/ESC-50/', '../data/.ESC-50.cache', TRAIN_FOLDS, VALIDATION_FOLDS,
                    TEST_FOLDS)
# esc50_real = OriginalESC_real('../data/ESC-50/', '../data/.ESC-50.cache', TRAIN_FOLDS, VALIDATION_FOLDS,
#                     TEST_FOLDS)

In [ ]:
#populate training from validation

esc50.train_data = esc50.validation_data
esc50.train_folds = esc50.validation_folds
esc50.train_segments = esc50.validation_segments

In [ ]:
import h5py 

#save out test data:
with h5py.File('testh5pyreal.h5', 'w') as test:
    test.create_dataset('label', data=np.array(esc50.test_data.meta['target']))
    test.create_dataset('data', data=esc50.test_data.X)
    
#save out train data:
with h5py.File('trainh5pyreal.h5', 'w') as train:
    train.create_dataset('label', data=np.array(esc50.train_data.meta['target']))
    train.create_dataset('data', data=esc50.train_data.X)
    

In [ ]:
esc50.test_data.X.shape

In [ ]:
# esc50.test_data.X.shape
# esc50.train
from pprint import pprint
pprint(vars(esc50))
# X -= self.data_mean
#             X /= self.data_std
# esc50.test_data.X.dtype
# esc50_real.test_data.X[0][0][0][100]




In [ ]:
#flatten data
# train = {'data': esc50.train_data.X, 'label': np.array(esc50.train_data.meta['target'])}
# test = {'data': esc50.test_data.X, 'label': np.array(esc50.test_data.meta['target'])}

In [ ]:
# train["data"].shape

In [ ]:
# train['label'].shape
# type(train['label'])
# n = np.array(esc50.test_data.meta['target'])
# np.array(train['label']['target'])
# train['data'].shape
# test['label']
print(esc50.test_data.meta['target'].shape)
print(esc50.train_data.meta['target'].shape)


In [ ]:
import h5py 

#save out test data:
# with h5py.File('testh5py.h5', 'w') as test:
#     test.create_dataset('label', np.array(esc50.test_data.meta['target']))
#     test.create_dataset('data', data=esc50.test_data.X)

#NOT WORKING DIMENSIONALITY ISSUE ON H5PY SAVE!



#save out test data:
with h5py.File('testh5py.h5', 'w') as test:
    test.create_dataset('label', data=np.array(esc50.test_data.meta['target']))
    test.create_dataset('data', data=esc50.test_data.X)
    

In [ ]:

#save out train data:
with h5py.File('trainh5py.h5', 'w') as train:
    train.create_dataset('label', data=np.array(esc50.train_data.meta['target']))
    train.create_dataset('data', data=esc50.train_data.X)
    
    

In [ ]:
#save train and test individually
import deepdish as dd
import numpy as np
dd.io.save('train.h5', train, compression=None)
dd.io.save('test.h5', test, compression=None)

In [ ]:
train['data'].shape

In [ ]:
# import h5py


# with h5py.File('data.h5', 'w') as hf:
#     hf.create_dataset('data', data=train['data'])
#     hf.create_dataset('label', data=train['label'])

In [ ]:
esc50

In [ ]:
# from pprint import pprint
# # pprint(vars(esc50))
# # pprint(vars(esc50))
# # pprint(vars(esc50_real))

# # esc50 = []
# # pprint(vars(esc50.test_data))
# # esc50.train_folds
# print(esc50.train_size)
# print(esc50.test_size)
# print(esc50.validation_size)
# print(esc50.test_data.X.shape)
# print(esc50.validation_data.X.shape)
# print(esc50_real.test_data.X.shape)
# print(esc50_real.validation_data.X.shape)

In [ ]:
# print(esc50.__dict__)
# print(type(esc50))
# for key, value in esc50.items() :
#     print (key)
# print(esc50.test_data)
# print(esc50.test_size)
# print(esc50.train_data)
# print(esc50.train_size)


# print(esc50.train_data.meta.shape)
# print(esc50.train_data.X.shape)

In [ ]:
# esc50.train_data
# 

In [ ]:
# import hdf5storage as hdf5storage

# hdf5storage.writes(esc50, filename='newdata.h5')

In [ ]:
# import deepdish as dd
# import numpy as np

# X = np.zeros((100, 3, 32, 32))
# y = np.zeros(100)

# dd.io.save('data.h5', esc50, compression=None)

In [ ]:
# esc50train = esc50.train_data
# esc50test = esc50.test_data

In [ ]:
# train = {'data': esc50.train_data.X, 'label': esc50.train_data.meta}
# test = {'data': esc50.test_data.X, 'label': esc50.test_data.meta}

In [ ]:
# train

In [ ]:
# esc50test.X.shape
# esc50test.meta.shape

# #save train and test individually
# dd.io.save('esc50train.h5', esc50train, compression=None)
# dd.io.save('esc50test.h5', esc50test, compression=None)

In [ ]:
# esc50 == esc50_real

In [ ]:
# #python dictionary checker
# def find(lst, key, value):
#     for i, dic in enumerate(lst):
#         if dic[key] == value:
#             return i
#     return -1

In [ ]:


# # esc50.test_data
# print(esc50.test_data.X.dtype)
# print(esc50.test_data.X.shape)

# print(esc50.test_data.X.dtype)
# print(esc50_real.test_data.X.shape)

In [ ]:
# esc

In [ ]:
#ATTEMPTS AT SHOWING A SPECTROGRAM:

In [ ]:
# import matplotlib.pyplot as plt
# import librosa
# plt.figure(figsize=(12,4))
# #path stuff:
# import sys
# sys.path.append('/Users/rileyedmunds/echonet/echonet')
# # %load '../echonet'
# # %load '../echonet/datasets/esc_original.py'


# specfile = esc50.work_dir + '5-9032-A-0.wav' + '.orig.spec.npy'.format(esc50.bands)

# audio = echonet.load_audio(self.data_dir + 'audio/' + row.filename, 22050)
# audio *= 1.0 / np.max(np.abs(audio))

# spec = librosa.feature.melspectrogram(audio, sr=22050, n_fft=1024,
#                                       hop_length=512, n_mels=self.bands)
# spec = librosa.logamplitude(spec)

# np.save(specfile, spec, allow_pickle=False)



# # Display the spectrogram on a mel scale
# # sample rate and hop length parameters are used to render the time axis
# librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')

# # Put a descriptive title on the plot
# plt.title('mel power spectrogram')

# # draw a color bar
# plt.colorbar(format='%+02.0f dB')


# # Make the figure layout compact
# plt.tight_layout()

# plot.show()